## Fiducial cut for one .root file.

In [1]:
import ROOT
import numpy as np

Welcome to JupyROOT 6.28/00


In [12]:
class XCalHitsDataset:
    
    # Parameters specific to your setup
    scoringPlaneZ = 239.9985
    ecalFaceZ = 247.932
    cell_radius = 5.0
    fiducial_mode = True
    
    def __init__(self, filename):
        self.filename = filename
        self.tfile = ROOT.TFile(self.filename)
        self.ttree = self.tfile.Get('skimmed_events')
        self._load_cellMap()
        
    def _load_cellMap(self, version='v14'):
        self._cellMap = {}
        if version=='v13' or version=='v14':
            zd = np.loadtxt('/home/aether_zhou_2023/LDMX-scripts/GraphNet/data/%s/layer.txt' % version)
            # relative directory changed to absolute due to error:
            # FileNotFoundError: data/v14/layer.txt not found.
            self._layerZs = {round(zd[i]):i for i in range(len(zd))}
        if self.fiducial_mode:
            for i, x, y in np.loadtxt('/home/aether_zhou_2023/LDMX-scripts/GraphNet/data/%s/cellmodule.txt' % version):
                self._cellMap[i] = (x, y)
            self._cells = np.array(list(self._cellMap.values()))
        print("Loaded geometry info")

    # Calculate projection for fiducial test
    def projection(self, Recoilx, Recoily, Recoilz, RPx, RPy, RPz, HitZ):
        x_final = Recoilx + RPx/RPz*(HitZ - Recoilz) if RPz != 0 else 0
        y_final = Recoily + RPy/RPz*(HitZ - Recoilz) if RPy != 0 else 0
        return (x_final, y_final)

    # Calculate 2D Euclidean distance between cell center and hit
    def dist(self, cell, fXY):
        return np.sqrt((cell[0] - fXY[0])**2 + (cell[1] - fXY[1])**2)
    
    def compute_fiducial_cut(self):
        # load relevant leaves
        recoilX_leaf = self.ttree.GetLeaf('recoilX_')
        recoilY_leaf = self.ttree.GetLeaf('recoilY_')
        recoilPx_leaf = self.ttree.GetLeaf('recoilPx_')
        recoilPy_leaf = self.ttree.GetLeaf('recoilPy_')
        recoilPz_leaf = self.ttree.GetLeaf('recoilPz_')

        # compute cut array
        N = self.ttree.GetEntries()
        f_cut = np.zeros(N, dtype=bool)
        for i in range(N):
            self.ttree.GetEntry(i)
            fiducial = False #0
            fXY = self.projection(recoilX_leaf.GetValue(), recoilY_leaf.GetValue(), self.scoringPlaneZ, 
                                  recoilPx_leaf.GetValue(), recoilPy_leaf.GetValue(), recoilPz_leaf.GetValue(), self.ecalFaceZ)
            if not all(val == -9999 for val in [recoilX_leaf.GetValue(), recoilY_leaf.GetValue(), 
                                                recoilPx_leaf.GetValue(), recoilPy_leaf.GetValue(), recoilPz_leaf.GetValue()]):
                for cell in self._cells:
                    celldis = self.dist(cell, fXY)
                    if celldis <= self.cell_radius:
                        fiducial = True #1
                        break
            f_cut[i] = fiducial
        print("Fiducial cut computation completed.")
        return f_cut
        

In [13]:
# Path to your .root file
filename = "/home/duncansw/GraphNet_input/v14/8gev/v3_tskim/XCal_total/v14_8gev_0.1_XCal_total_4.root"

In [14]:
# Create an instance of your class and run the test
dataset = XCalHitsDataset(filename)
f_cut = dataset.compute_fiducial_cut()
print(f"Fiducial cut flags: {len(f_cut)}")

Loaded geometry info
Fiducial cut computation completed.
Fiducial cut flags: 18461


In [15]:
oncell = 0
offcell = 0
fid_events = []
non_fid_events = []
for index in range(0,len(f_cut)):
    if f_cut[index] == 1:
        oncell += 1
        fid_events.append(index)
    elif f_cut[index] == 0:
        offcell += 1
        non_fid_events.append(index)
print('oncell: {}\noffcell: {}'.format(oncell,offcell))

oncell: 11955
offcell: 6506


In [16]:
non_fid_events[10]

33

## test

In [7]:
tfile = ROOT.TFile(filename)
ttree = tfile.Get('skimmed_events')
recoilX_leaf = ttree.GetLeaf('recoilX_')

In [8]:
ttree.GetEntries() # total event number

18461

In [9]:
ttree.GetEntry(2)
# ttree.Show()
recoilX_leaf.GetValue()

-16.1285400390625

In [10]:
recoilX_leaf.GetValue(2)

346890960896.0